# Gradient and Spin Echo Pulse Sequences

There are two broad classes of pulse sequences used: gradient-echo and spin-echo methods.  Gradient-echo sequences use a simpler pulse and acquire acquisition, while spin-echo sequences use an additional RF refocusing pulse that eliminates the effects of off-resonance that are often undesirable.

## Learning Goals

1. Describe how various types of MRI contrast are created
    * Describe the difference between gradient and spin echo contrast
1. Understand the most popular pulse sequences and their acronyms
    * Identify gradient echo and spin echo pulse sequences
1. Identify artifacts and how to mitigate them
    * Understand the effects of off-resonance and T2* on contrast


In [ ]:
% setup MRI-education-resources path and requirements
cd ../
startup

loading image
loading signal


## Gradient Echo Pulse Sequence (GE or GRE)

The building block of a gradient-echo (GE) pulse sequence includes an RF excitation pulse followed by imaging gradients.  A complete 2D imaging sequence is

![Gradient echo pulse sequence](images/pulse_sequence-diagram-all.png)

It is called a "gradient-echo (GE)" since the frequency encoding imaging gradients, shown on $G_X$, are refocused or canceled out at the echo time (more in Spatial Encoding section).  Another name for these sequences is a "gradient-recalled echo (GRE)".

And using a simplified diagram for the gradient echo sequence:

![Simplified Gradient echo pulse sequence](images/pulse_sequence-diagram-simple.png)


## Off-resonance

Ideally, the magnetic field is uniform in the absense of any applied gradients.  However, in practice there are unavoidable variations in the magnetic field that lead to changes in resonance frequency.  These magnetic field variations are referred to as "off-resonance", and can be represented as

$$\Delta f_r(\vec{r})$$

This typically includes imperfections in the main $B_0$ magnetic as well as subject-specific changes in the magnetic field that are largely due to magnetic susceptibility effects.

## Effects of Off-resonance and T2*

Ideally, the MRI signal decays with a time constant of $T_2$, the transverse decay rate.  However, in practice every imaging voxel contains off-resonance that leads to changes in the signal decay.  We characterize this change in signal decay with $T_2^*$.  

The smallest size we can measure with MRI is an imaging voxel, so our image measurements are a sum of the transverse magnetizations across the voxel, including both $T_2$ and off-resonance: 

$$\begin{align}
s_{\mathrm{voxel}}(t) & = \int_{\mathrm{voxel}} M_{XY}(\vec{r},0) \exp(-t/T_2(\vec{r})) \exp(-i 2 \pi \Delta f_r(\vec{r}))  \  d\vec{r} \\
& = M_{XY}(\vec{r},0) \exp(-t/T_2^*(\vec{r}))
\end{align}$$

(Note that any effects of applied gradients are removed during the image reconstruction process.)

The mechanism of this decay is that, across an imaging voxel, the transverse magnetizations across the voxel precess at slightly different rates.  This is called "de-phasing", and means that when the transverse magnetization is averaged across the voxel it's overall magnitude is reduced.

$T_2^*$ is an approximation of these dephasing effects, and depends on the voxel size and location.  It is often broken up as follows to separate out the additional dephasing rate, $1/T_2'$:

$$ \frac{1}{T_2^*} = \frac{1}{T_2}+\frac{1}{T_2'}$$

$$ T_2^* \leq T_2$$


In [ ]:
% Simulations of off-resonance and T2* decay

B0 = 1.5e3; % 1.5 T = 1500 mT
% using units of mT and ms for the bloch_rotate function
T2 = 80; % ms
T1 = Inf; % neglect T1

% create range of off-resonance in a voxel
Nvoxel = 8^2;
off_resonance_range = 0.5e-6; % off resonance range, in ppm
dBZ = linspace(-off_resonance_range,off_resonance_range, Nvoxel); % assumes a uniform distribution
dBZ = (rand(Nvoxel) -0.5 )* off_resonance_range;  % Uniform distribution
dBZ = randn(Nvoxel) * off_resonance_range;  % Gaussian distribution

spinecho_flag = 1;


% start after RF excitation
Mstart = [1,0,0].'; 

dt = .1; % ms
N = 300;
t = [1:N]*dt;
Mall = zeros(3,N,Nvoxel);
Mall(:,1,:) = repmat(Mstart, [1 1 Nvoxel]);
for INvoxel = 1:Nvoxel
    Bvoxel = [0 0 dBZ(Nvoxel)]; % off-resonance, in rotating frame
    for It = 1:N-1
        if spinecho_flag && It == N/2
            Mall(2,It,INvoxel) = -Mall(2,It,INvoxel); % 180-pulse to invert My
        end
        Mall(:,It+1,INvoxel) = bloch_relax(  bloch_rotate(Mall(:,It,INvoxel),dt,Bvoxel), dt, 1, T1, T2 );
    end
end

Mall_voxel = sum(Mall, 3);

[x y] = meshgrid(1:sqrt(Nvoxel));
Splot = 0.5;

for It = 1:N
    Mxy_plot = Mall(1,It,:) + 1i*Mall(2,It,:);
    Mxy_plot = reshape(Mxy_plot, [sqrt(Nvoxel) sqrt(Nvoxel)]);

    subplot(3,1,1)
    quiver(x-real(Mxy_plot)*Splot/2,y-imag(Mxy_plot)*Splot/2,...
            real(Mxy_plot), imag(Mxy_plot), ...
            Splot)

    subplot(3,1,2)
    Mxy_voxel = sum(sum(Mxy_plot));
    quiver(-real(Mxy_voxel)*Splot/2,-imag(Mxy_voxel)*Splot/2,...
            real(Mxy_voxel), imag(Mxy_voxel), ...
            Splot)

    subplot(3,1,3)
    plot(t(1:It), Mall_voxel(1,1:It), t(1:It), Mall_voxel(2,1:It),'--')
    xlim([0 max(t)])

end


## Refocusing off-resonance

### Simulation for Visualization of refocusing off-resonance

http://drcmr.dk/BlochSimulator/
Scene: Inhomogeneity
Apply 90x hard, then 180y hard
Apply 90x hard, then 180x hard
Apply 90x hard, then repeated 180s

## Spin Echo Pulse Sequence (SE)

The building block of a spin-echo (SE) pulse sequence has an additional 180-degree refocusing pulse between the excitation and data acquisition in order to refocus the effects of off-resonance and create pure T2-weighting:

![Spin echo pulse sequence](images/pulse_sequence-diagram-spinecho-all.png)

It is called a "gradient-echo (GE)" since the spins are all refocused at the echo time, TE.

This is a simplified diagram for the spin echo sequence:

![Simplified Spin echo pulse sequence](images/pulse_sequence-diagram-spinecho-simple.png)

In [ ]:
% Spin-echo simulation